## Conversion to fif: BJH039

This script takes the raw, original data file we recieved from WashU and converts it to a fif file with the photodiode as a new stim channel


Just need to change fields in `prep_paths`

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
from tabulate import tabulate
import IPython
import seaborn as sns

In [2]:
## prep paths

subject = 'BJH039'
orig_data_fi = '/home/brooke/knight_server/remote/WashU/data/PacmanTask/BJH039/PacmanTask/ECOG001/ECOGS001R01.dat'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}/ieeg"

In [3]:
## load data

with b2k.BCI2kReader(orig_data_fi) as data: #opens a stream to the dat file
    my_signals = data.signals
    my_states = data.states
    my_params = data.parameters

In [5]:
## check structure if necessary

my_states
# my_params

{'WebcamFrame0': array([[    0,     0,     0, ..., 27297, 27298, 27298]], dtype=int32),
 'WebcamFrame1': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'WebcamFrame2': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'WebcamFrame3': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'KeyDown': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'KeyUp': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'GhostLocation': array([[  0,   0,   0, ..., 136, 136, 136]], dtype=int32),
 'UserLocation': array([[ 0,  0,  0, ..., 74, 74, 74]], dtype=int32),
 'Direction': array([[4, 4, 4, ..., 4, 4, 4]], dtype=int32),
 'Biscuit1': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'Biscuit2': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'Biscuit3': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'Biscuit4': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'Biscuit5': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'Attack': array([[0, 0, 0, ..., 0, 0, 0]], dtype=int32),
 'Chase': array([[0, 0, 0, ..., 0, 0, 

In [6]:
## create info field 

info = mne.create_info(ch_names = my_params['ChannelNames'],
                       sfreq = my_params['SamplingRate'],
                       ch_types = 'seeg')

info

<Info | 7 non-empty values
 bads: []
 ch_names: AR1, AR2, AR3, AR4, REF1, REF2, AR5, AR6, AR7, AR8, AR9, AR10, ...
 chs: 256 sEEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 nchan: 256
 projs: []
 sfreq: 2000.0 Hz
>

In [7]:
my_params['ChannelNames']

['AR1',
 'AR2',
 'AR3',
 'AR4',
 'REF1',
 'REF2',
 'AR5',
 'AR6',
 'AR7',
 'AR8',
 'AR9',
 'AR10',
 'AR11',
 'AR12',
 'BR1',
 'BR2',
 'BR3',
 'BR4',
 'BR5',
 'BR6',
 'BR7',
 'BR8',
 'BR9',
 'BR10',
 'BR11',
 'BR12',
 'BR13',
 'BR14',
 'CR1',
 'CR2',
 'CR3',
 'CR4',
 'CR5',
 'CR6',
 'CR7',
 'CR8',
 'CR9',
 'CR10',
 'DR1',
 'DR2',
 'DR3',
 'DR4',
 'DR5',
 'DR6',
 'DR7',
 'DR8',
 'DR9',
 'DR10',
 'DR11',
 'DR12',
 'ER1',
 'ER2',
 'ER3',
 'ER4',
 'ER5',
 'ER6',
 'ER7',
 'ER8',
 'ER9',
 'ER10',
 'ER11',
 'ER12',
 'ER13',
 'ER14',
 'FR1',
 'FR2',
 'FR3',
 'FR4',
 'FR5',
 'FR6',
 'FR7',
 'FR8',
 'FR9',
 'FR10',
 'GR1',
 'GR2',
 'GR3',
 'GR4',
 'GR5',
 'GR6',
 'GR7',
 'GR8',
 'GR9',
 'GR10',
 'GR11',
 'GR12',
 'GR13',
 'GR14',
 'HR1',
 'HR2',
 'HR3',
 'HR4',
 'HR5',
 'HR6',
 'HR7',
 'HR8',
 'HR9',
 'HR10',
 'HR11',
 'HR12',
 'IR1',
 'IR2',
 'IR3',
 'IR4',
 'IR5',
 'IR6',
 'IR7',
 'IR8',
 'IR9',
 'IR10',
 'IR11',
 'IR12',
 'IR13',
 'IR14',
 'JR1',
 'JR2',
 'JR3',
 'JR4',
 'JR5',
 'JR6',
 'JR7',

In [8]:
## load data
orig_data_fi_a = '/home/brooke/knight_server/remote/WashU/data/AlexisSingingTask/BJH039/SingingTask/ECOG001/ECOGS001R01.dat'
with b2k.BCI2kReader(orig_data_fi_a) as alexis_data: #opens a stream to the dat file
    alexis_my_signals = alexis_data.signals
    alexis_my_states = alexis_data.states
    alexis_my_params = alexis_data.parameters

/home/brooke/anaconda3/envs/ieeg_analysis/lib/python3.9/site-packages/BCI2kReader/FileReader.py:270: RuntimeWarning: invalid value encountered in cast
  states[statename] = numpy.array(sd['mult'] * numpy.asmatrix(rawstates[sd['slice'], :] & sd['mask']),


In [11]:
alexis_my_params['ChannelNames']

['AR1',
 'AR2',
 'AR3',
 'AR4',
 'REF1',
 'REF2',
 'AR5',
 'AR6',
 'AR7',
 'AR8',
 'AR9',
 'AR10',
 'AR11',
 'AR12',
 'BR1',
 'BR2',
 'BR3',
 'BR4',
 'BR5',
 'BR6',
 'BR7',
 'BR8',
 'BR9',
 'BR10',
 'BR11',
 'BR12',
 'BR13',
 'BR14',
 'CR1',
 'CR2',
 'CR3',
 'CR4',
 'CR5',
 'CR6',
 'CR7',
 'CR8',
 'CR9',
 'CR10',
 'DR1',
 'DR2',
 'DR3',
 'DR4',
 'DR5',
 'DR6',
 'DR7',
 'DR8',
 'DR9',
 'DR10',
 'DR11',
 'DR12',
 'ER1',
 'ER2',
 'ER3',
 'ER4',
 'ER5',
 'ER6',
 'ER7',
 'ER8',
 'ER9',
 'ER10',
 'ER11',
 'ER12',
 'ER13',
 'ER14',
 'FR1',
 'FR2',
 'FR3',
 'FR4',
 'FR5',
 'FR6',
 'FR7',
 'FR8',
 'FR9',
 'FR10',
 'GR1',
 'GR2',
 'GR3',
 'GR4',
 'GR5',
 'GR6',
 'GR7',
 'GR8',
 'GR9',
 'GR10',
 'GR11',
 'GR12',
 'GR13',
 'GR14',
 'HR1',
 'HR2',
 'HR3',
 'HR4',
 'HR5',
 'HR6',
 'HR7',
 'HR8',
 'HR9',
 'HR10',
 'HR11',
 'HR12',
 'IR1',
 'IR2',
 'IR3',
 'IR4',
 'IR5',
 'IR6',
 'IR7',
 'IR8',
 'IR9',
 'IR10',
 'IR11',
 'IR12',
 'IR13',
 'IR14',
 'JR1',
 'JR2',
 'JR3',
 'JR4',
 'JR5',
 'JR6',
 'JR7',

In [12]:
tmp = ['AR1',
 'AR2',
 'AR3',
 'AR4',
 'REF1',
 'REF2',
 'AR5',
 'AR6',
 'AR7',
 'AR8',
 'AR9',
 'AR10',
 'AR11',
 'AR12',
 'BR1',
 'BR2',
 'BR3',
 'BR4',
 'BR5',
 'BR6',
 'BR7',
 'BR8',
 'BR9',
 'BR10',
 'BR11',
 'BR12',
 'BR13',
 'BR14',
 'CR1',
 'CR2',
 'CR3',
 'CR4',
 'CR5',
 'CR6',
 'CR7',
 'CR8',
 'CR9',
 'CR10',
 'DR1',
 'DR2',
 'DR3',
 'DR4',
 'DR5',
 'DR6',
 'DR7',
 'DR8',
 'DR9',
 'DR10',
 'DR11',
 'DR12',
 'ER1',
 'ER2',
 'ER3',
 'ER4',
 'ER5',
 'ER6',
 'ER7',
 'ER8',
 'ER9',
 'ER10',
 'ER11',
 'ER12',
 'ER13',
 'ER14',
 'FR1',
 'FR2',
 'FR3',
 'FR4',
 'FR5',
 'FR6',
 'FR7',
 'FR8',
 'FR9',
 'FR10',
 'GR1',
 'GR2',
 'GR3',
 'GR4',
 'GR5',
 'GR6',
 'GR7',
 'GR8',
 'GR9',
 'GR10',
 'GR11',
 'GR12',
 'GR13',
 'GR14',
 'HR1',
 'HR2',
 'HR3',
 'HR4',
 'HR5',
 'HR6',
 'HR7',
 'HR8',
 'HR9',
 'HR10',
 'HR11',
 'HR12',
 'IR1',
 'IR2',
 'IR3',
 'IR4',
 'IR5',
 'IR6',
 'IR7',
 'IR8',
 'IR9',
 'IR10',
 'IR11',
 'IR12',
 'IR13',
 'IR14',
 'JR1',
 'JR2',
 'JR3',
 'JR4',
 'JR5',
 'JR6',
 'JR7',
 'JR8',
 'KL1',
 'KL2',
 'KL3',
 'KL4',
 'KL5',
 'KL6',
 'KL7',
 'KL8',
 'KL9',
 'KL10',
 'KL11',
 'KL12',
 'LL1',
 'LL2',
 'LL3',
 'LL4',
 'LL5',
 'LL6',
 'LL7',
 'LL8',
 'LL9',
 'LL10',
 'LL11',
 'LL12',
 'ML1',
 'ML2',
 'ML3',
 'ML4',
 'ML5',
 'ML6',
 'ML7',
 'ML8',
 'ML9',
 'ML10',
 'ML11',
 'ML12']

len(tmp)

158

In [9]:
## create mne object
mne_df = mne.io.RawArray(my_signals, info)

Creating RawArray with float64 data, n_channels=256, n_times=3639800
    Range : 0 ... 3639799 =      0.000 ...  1819.899 secs
Ready.


In [10]:
## create stim channel and add it to mne object

info = mne.create_info(['STI'], my_params['SamplingRate'], ['stim'])
stim_raw = mne.io.RawArray( my_states['Trial_on_off'], info)
mne_df.add_channels([stim_raw], force_update_info=True)

Creating RawArray with float64 data, n_channels=1, n_times=3639800
    Range : 0 ... 3639799 =      0.000 ...  1819.899 secs
Ready.


<RawArray | 257 x 3639800 (1819.9 s), ~6.97 GB, data loaded>

In [9]:
## save it

mne_df.save(f"{raw_data_dir}/{subject}_raw_ieeg.fif", overwrite = False)

Writing /home/brooke/pacman/raw_data/BJH039/ieeg/BJH039_raw_ieeg.fif
Overwriting existing file.
Writing /home/brooke/pacman/raw_data/BJH039/ieeg/BJH039_raw_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/BJH039/ieeg/BJH039_raw_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/BJH039/ieeg/BJH039_raw_ieeg.fif
[done]
